In [ ]:
import pandas as pd
import os
from datetime import datetime
import mydashtools as dt
import plotly.express as px
pd.options.plotting.backend = "plotly"
if dt.is_notebook():
    __file__ = "notebook/"

zelty = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)
zelty['date'] = pd.to_datetime(zelty['date'])
if dt.is_notebook():
    print(zelty.head())

events = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../events/barco_events_all.csv", sep=";",decimal=',',low_memory=False, index_col=False)
events['date'] = pd.to_datetime(events['date'])
events.set_index('date', inplace=True)
events_by_day = events.resample('D')['titre'].apply(' / '.join).reset_index()
events_by_day['date'] = events_by_day['date'].apply(lambda x: str(x)[0:10])
events_by_day.set_index('date', inplace=True)
d2e = events_by_day[events_by_day['titre'] != '']['titre'].to_dict()

In [ ]:
adhésions = zelty[zelty['category'] == 'Adhésion'].groupby('date')[['price']].count()

In [ ]:
limit_date = datetime.now().replace(month=datetime.now().month - 1)
adhésions_last_month = adhésions[adhésions.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
adhésions_last_month = adhésions_last_month.resample('D').sum().rename(columns={'price': 'Nb d\'adhésion'})
adhésions_last_month['évènement'] = adhésions_last_month.index.strftime('%Y-%m-%d').map(d2e).fillna('<aucun>')

fig = px.bar(adhésions_last_month, y=['Nb d\'adhésion'], hover_data=['évènement'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'adhesions_01_adhesion_des_derniers_jours')

In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
adhésions_top_year = adhésions[adhésions.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
adhésions_top_year = adhésions_top_year.resample('D').sum().rename(columns={'price': 'Nb d\'adhésion'})
if dt.is_notebook():
    print(adhésions_top_year.sort_values(by='Nb d\'adhésion', ascending=False).head(20))

In [ ]:
adhésions_by_month = adhésions.resample('M').sum().rename(columns={'price': 'Nb d\'adhésion'})
adhésions_by_month['Année'] = adhésions_by_month.index.year
adhésions_by_month = adhésions_by_month.groupby(['date','Année']).sum().unstack('Année').fillna(0)
adhésions_by_month.columns = adhésions_by_month.columns.droplevel(0)

fig = px.bar(adhésions_by_month)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'adhesions_02_adhesion_par_mois')

In [ ]:
#adhésions_by_year.plot(kind="bar")

In [ ]:
adhésions_by_year = adhésions.resample('Y').sum().rename(columns={'price': 'Nb d\'adhésion'})
adhésions_by_year.index = adhésions_by_year.index.year

adhésions_a_date = adhésions.copy()
adhésions_a_date['a_date'] = adhésions_a_date.index.month*100+adhésions_a_date.index.day
adhésions_a_date = adhésions_a_date[adhésions_a_date['a_date'] < datetime.now().month*100+datetime.now().day]
del adhésions_a_date['a_date']
adhésions_a_date = adhésions_a_date.resample('Y').sum().rename(columns={'price': 'Nb d\'adhésion'})
adhésions_a_date.index = adhésions_a_date.index.year
libelle_a_date = "À date du %02d/%02d" % (datetime.now().day, datetime.now().month)
adhésions_by_year[libelle_a_date] = adhésions_a_date['Nb d\'adhésion']
#adhésions_by_year.histogram()
adhésions_by_year.fillna(0, inplace=True)
adhésions_by_year['toute l\'année'] = adhésions_by_year['Nb d\'adhésion']# - adhésions_by_year[libelle_a_date]
del adhésions_by_year['Nb d\'adhésion']
del adhésions_by_year['toute l\'année'][max(adhésions_by_year.index)]

import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Bar(name=libelle_a_date, x=adhésions_by_year.index, y=adhésions_by_year[libelle_a_date]),
    go.Bar(name='Toute l\'année', x=adhésions_by_year.index, y=adhésions_by_year['toute l\'année'])
])
# Change the bar mode
fig.update_layout(separators=', ')
fig.update_yaxes(tickformat=",d")

dt.fig_save_or_show(fig, 'adhesions_03_adhesion_par_an')